<a href="https://colab.research.google.com/github/krvicky/open_spiel/blob/main/Openspiel_CFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required dependencies
!pip install open-spiel  # This might take some time

# Import necessary libraries
from open_spiel.python.algorithms import cfr
from open_spiel.python.algorithms import exploitability
import pyspiel

# Define configuration parameters directly
my_iterations = 100
my_game = "kuhn_poker"
players = 2
print_freq = 10

# Load the game using the specified game name
game = pyspiel.load_game(my_game, {"players": players})
cfr_solver = cfr.CFRSolver(game)

# Run your iterations
for i in range(my_iterations):
    cfr_solver.evaluate_and_update_policy()
    if i % print_freq == 0:
        conv = exploitability.exploitability(game, cfr_solver.average_policy())
        print("Iteration {} exploitability {}".format(i, conv))


Iteration 0 exploitability 0.45833333333333326
Iteration 10 exploitability 0.06046924690611866
Iteration 20 exploitability 0.039914275345009825
Iteration 30 exploitability 0.024167348753902612
Iteration 40 exploitability 0.020517348345035824
Iteration 50 exploitability 0.014479024570810684
Iteration 60 exploitability 0.014003542854660017
Iteration 70 exploitability 0.011778275229671564
Iteration 80 exploitability 0.010102437916053336
Iteration 90 exploitability 0.00983365028965788
